# Challenge 1

### Imports

In [ ]:
import requests
import os
import zipfile
from bs4 import BeautifulSoup

### Log in to hackthissite.org

In [ ]:
username = "FriedrichMaximilian"
password = os.getenv('HTS_PASSWORD')
if not password:
    print("Password not found in environment variables. Please set 'HTS_PASSWORD'.")
    exit()

login_url = "https://www.hackthissite.org/user/login"
mission_url = "https://www.hackthissite.org/missions/prog/1/"
submit_url = "https://www.hackthissite.org/missions/prog/1/index.php"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Referer': login_url,
}

session = requests.Session()

login_page_response = session.get(login_url, headers=headers)

if login_page_response.status_code == 200:
    print("Login page fetched successfully!")
else:
    print(f"Failed to fetch login page with status code: {login_page_response.status_code}")
    exit()

login_data = {
    'username': username,
    'password': password
}

login_response = session.post(login_url, data=login_data, headers=headers)

if "hello" in login_response.text.lower():
    print("Login successful!")
else:
    print("Login failed.")

### List of scrambled words

In [ ]:
mission_page = session.get(mission_url)

if mission_page.status_code == 200:
    soup = BeautifulSoup(mission_page.text, 'html.parser')
    scrambled_section = soup.find_all('li')  # Modify this based on actual HTML structure
    scrambled_words = [word.text for word in scrambled_section]
    print("Scrambled Words:", scrambled_words[-10:])
else:
    print("Failed to fetch the mission page.")

### Download wordlist

In [ ]:
url = "https://www.hackthissite.org/missions/prog/1/wordlist.zip"
file_name = "wordlist.zip"

if not os.path.exists(file_name):
    
    response = requests.get(url, stream=True)
    
    if response.status_code == 200:
        with open(file_name, "wb") as file:
            for chunk in response.iter_content(chunk_size=1024):
                file.write(chunk)
        print(f"File downloaded successfully as {file_name}")
    else:
        print(f"Failed to download file. HTTP Status Code: {response.status_code}")
else:
    print(f"File '{file_name}' already exists. Skipping download.")


### Unzip file

In [ ]:
extract_folder = "wordlist"
if not os.path.exists(extract_folder):
    with zipfile.ZipFile(file_name, 'r') as zip_ref:
        zip_ref.extractall(extract_folder)
        print(f"File extracted to {extract_folder}")
else:
    print(f"Folder '{extract_folder}' already exists. Skipping extraction.")

### Read the words

In [ ]:
# Open and read the extracted file into a list, line by line
word_list = []
wordlist_path = os.path.join(extract_folder, "wordlist.txt")
if os.path.exists(wordlist_path):
    with open(wordlist_path, "r", encoding="utf-8") as file:
        for line in file:
            word_list.append(line.strip())
            # print(line.strip())  # Process each line as needed
else:
    print(f"Extracted wordlist file '{wordlist_path}' not found.")
# print(word_list)

### Finally, unscramble the words

In [ ]:
scrambled_words = scrambled_words[-10:]
word_list = word_list
word_list.sort()
print(scrambled_words, "\n")
# print(word_list)

# restructure the word list according to length of words
d_len = {}

lengths = []
for word in word_list:
    if len(word) not in d_len:
        d_len[len(word)] = [word]
        lengths.append(len(word))
    else:
        d_len[len(word)].append(word)
# print(lengths, "\n")
# print(d_len)
    

ans = []
for scrambled in scrambled_words:
    l = len(scrambled)
    words = d_len[l]
    candidates = []
    i = 0
    candidates = [x for x in words if scrambled[0] in x]
    for ch in scrambled:
        if i == 0:
            i += 1
            continue
        else:
            candidates = [x for x in candidates if ch in x]
    if len(candidates) > 1:
        for can in candidates:
            if sorted(scrambled) == sorted(can):
                candidates = [can]
                break
    # print(candidates)
    # print(scrambled)
    
    ans += candidates
print(len(ans), len(scrambled_words))
ans = ",".join(ans)
print(type(ans), ans)

### Submit solution

In [ ]:
payload = {
    "solution": ans, 
    "submitbutton": "submit"  
}
    
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
    "Referer": "https://www.hackthissite.org/missions/prog/1/"
}

response = session.post(mission_url, data=payload, headers=headers)

if "correct" in response.text.lower():
    print("Submission successful!")
elif "incorrect" in response.text.lower():
    print("Submission incorrect. Try again.")
else:
    print("Could not determine submission result.")

# Print response for debugging
# print(response.text)